In [1]:
import os
import sys

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np


import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.models import Model
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ThresholdedReLU

from skimage.io import imsave, imread
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from skimage.transform import resize

from keras.optimizers import Adam
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 7

Using TensorFlow backend.


In [2]:
#Set train and test paths
TRAIN_PATH = "/Users/sangeethags/Desktop/SEM8/AI/Assignment1/assignment2/dataset/others/"
TEST_PATH = "/Users/sangeethags/Desktop/SEM8/AI/Assignment1/assignment2/dataset/melanoma/"

# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[2]
test_ids = next(os.walk(TEST_PATH))[2]

In [3]:
# Get and resize train images and masks
not_infected_img = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
not_infected_lbl = np.zeros((len(train_ids), 1), dtype=np.bool)
print('Getting and resizing not infected images ... ')
sys.stdout.flush()
for n, img_file in tqdm(enumerate(train_ids), total=len(train_ids)):
    img = imread(TRAIN_PATH + img_file)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    not_infected_img[n] = img
    not_infected_lbl[n] = 0;
    
# Get and resize test images and masks
infected_img = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
infected_lbl = np.zeros((len(test_ids), 1), dtype=np.bool)
#sizes_test = []
print('Getting and resizing infected images ... ')
sys.stdout.flush()
for n, img_file in tqdm(enumerate(test_ids), total=len(test_ids)):
    img = imread(TEST_PATH + img_file)[:,:,:IMG_CHANNELS]
    #sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    infected_img[n] = img
    infected_lbl[n] = 1;

Getting and resizing not infected images ... 


100%|██████████| 1626/1626 [20:47<00:00,  1.30it/s]


Getting and resizing infected images ... 


100%|██████████| 374/374 [04:40<00:00,  1.33it/s]


In [4]:
class0len = len(not_infected_img)
class1len = len(infected_img)

#Append infected and non-infected into one array, shuffle them, then split into train and test
all_images = np.zeros((len(train_ids)+len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
all_labels = np.zeros((len(train_ids)+len(test_ids), 1), dtype=np.bool)

all_images[0:len(train_ids)] = not_infected_img
all_labels[0:len(train_ids)] = not_infected_lbl
all_images[len(train_ids):] = infected_img
all_labels[len(train_ids):] = infected_lbl

(X_train, X_test, Y_train, Y_test) = train_test_split(all_images,all_labels, test_size=0.25, random_state=42, shuffle = True)

In [6]:
from keras.models import Sequential

Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)

        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
        
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
 
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
 
        # return the constructed network architecture
        return model 

model = LeNet.build(width=128, height=128, depth=3, classes=2)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])

print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 20)      1520      
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 20)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 50)        25050     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 50)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 50)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
__________

In [7]:
classifier = model.fit(X_train,Y_train, batch_size = 10, epochs = 30, verbose = True, validation_data = (X_test, Y_test), class_weight = {0:1, 1:class0len/class1len})
Y_pred = model.predict(X_test,verbose="True")
Y_classes = Y_pred.argmax(axis=-1)
Y_pred = Y_pred.round()

print('F1 =', f1_score(testY, Y_pred, average=None))
testY = testY.argmax(1)
Y_pred = Y_pred.argmax(1)

tn, fp, fn, tp = confusion_matrix(testY, Y_pred, labels=None, sample_weight=None).ravel()
cm = [[tp, fn], [fp, tn]]
print('Confusion matrix = ', cm)
print('precision_score = ', precision_score(testY, Y_pred, pos_label=1, average='binary'))
print('recall_score = ', recall_score(testY, Y_pred, pos_label=1, average='binary'))

Train on 1500 samples, validate on 500 samples
Epoch 1/30
1500/1500 [==============================] - 2s 1ms/step - loss: 1.1254 - acc: 0.5940 - val_loss: 0.6870 - val_acc: 0.8060
Epoch 2/30
1500/1500 [==============================] - 1s 968us/step - loss: 1.1232 - acc: 0.5353 - val_loss: 0.6640 - val_acc: 0.8080
Epoch 3/30
1500/1500 [==============================] - 1s 817us/step - loss: 1.1213 - acc: 0.6193 - val_loss: 0.7033 - val_acc: 0.3660
Epoch 4/30
1500/1500 [==============================] - 1s 800us/step - loss: 1.1148 - acc: 0.5007 - val_loss: 0.5168 - val_acc: 0.8080
Epoch 5/30
1500/1500 [==============================] - 1s 828us/step - loss: 1.1091 - acc: 0.5753 - val_loss: 0.7339 - val_acc: 0.2200
Epoch 6/30
1500/1500 [==============================] - 1s 792us/step - loss: 1.1108 - acc: 0.4687 - val_loss: 0.5439 - val_acc: 0.7940
Epoch 7/30
1500/1500 [==============================] - 1s 788us/step - loss: 1.0904 - acc: 0.5653 - val_loss: 0.6673 - val_acc: 0.5300
Epo